# Optimisation of LCA models

* Minimise environment impact and/or cost of a network of processes 
* OpenLCA for lifecycle assessment data
* Pyomo for mathematical optimisation
* Jupyter notebooks for prototyping
* Qt GUI for configuration and solution

# OpenLCA

* Java application built by Greendata
* Front-end to life-cycle database of processes and flows
* Ecoinvent system database has ~18000 processes ~7000 flows
* System processes broken down into elementary flows
* Contains methodologies to calculate impact of system
* Cannot feasibly optimise over multiple alternatives

# Pyomo

* An algebraic modelling langugage like GAMS, AIMMS
* Implemented as an object model in Python package `pyomo`
* Eases translation of mathematical optimisation problem to actual implementation
* Optimisation problems are formulated by building concrete or abstract model


# Mola

* Python package builds optimisation models using openLCA data
* `mola.dataview`: SQL queries to query SQLite database of openLCA data (Ecoinvent system db) 
* `mola.specificationv5`: `Specification` classes each containing abstract Pyomo model
* `mola.build`: configure, populate, solve optimisation problem

## Specification

Decorates a `pyomo` abstract model so that we can build an interface using introspection.

```
class Specification():
    user_defined_sets = {'A': {'doc': 'test set', 'lookup'=False}}
    user_defined_parameters = {'Q': {'index': ['A'], 'doc'='test parameters'}}

    def __init__():
        self.abstract_model = AbstractModel()
        # build user defined and db parameters
        
    def populate(json_files): # populates abstract model
        dp = DataPortal()
        
```

# MolaQT

Front-end to Mola using `pyqt`. Built around a `Controller` object for each model

```
class Controller(QWidget):

  def __init__(spec):
      self.spec = spec
      self.sets = spec.get_default_sets()
      self.parameters = spec.get_default_parameters()
      self.lookup = LookupTables(self.db) # if required
  
  def get_config(self): # get current state
```

## StandardController

Aims to Mirror Pyomo modelling language using a collection of QtWidgets

```
class StandardController(Controller):

  def __init__(spec):
       self.sets_editor = SetsEditor(self.sets, self.lookup)
       self.parameters_editor = ParametersEditor(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
  
```

## CustomController

High-level widgets tied to a particular model and workflow. Hardcodes the model to the UI.

```
class StandardController(Controller):

  def __init__(spec):
       self.objectives = ObjectivesWidget(self.sets, self.lookup)
       self.ProcessFlow = ProcessFlow(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
 ```

## DashboardController

* Not yet implemented
* No need to use tabbed interface
* Could create a single dashboard if model is sufficiently simple
* Incorporate or export output

# Demos

* General overview: Github, DB, General Specification, Models
* AIMMS tutorial problem: interface and model construction
* Orange production and transport: openLCA data (StandardController)
* Orange production and transport: openLCA data (CustomController)

# Development work

* Tasks: pyomo.network - MultiTaskController, ScheduleController, DashboardController
* Storage: AIMMS advanced example, Sargent papers, STNs, RTNs
* Output: current module to convert pyomo output to DataFrames for export, graphing
* More sophistical openLCA models e.g. oil to diesel conversion
* Dynamic model generation?